In [ ]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt

# AWS Credentials and Region
aws_access_key_id = 'YOUR_AWS_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_AWS_SECRET_ACCESS_KEY'
region_name = 'us-west-2'  # Replace with your desired AWS region

# Initialize CloudWatch client
cloudwatch = boto3.client('cloudwatch', aws_access_key_id=aws_access_key_id,
                          aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Define the metric names and dimensions for comparison
metric_names = ['CPUUtilization', 'ReadThrottleEvents', 'WriteThrottleEvents']
dimensions = [{'Name': 'DBInstanceIdentifier', 'Value': 'your-db-instance-id'}]  # Replace with your DB instance ID

# Retrieve metrics data from CloudWatch
data = []
for metric_name in metric_names:
    response = cloudwatch.get_metric_data(
        MetricDataQueries=[
            {
                'Id': 'm1',
                'MetricStat': {
                    'Metric': {
                        'Namespace': 'AWS/RDS',
                        'MetricName': metric_name,
                        'Dimensions': dimensions
                    },
                    'Period': 3600,  # Adjust the period as per your requirements
                    'Stat': 'Average',  # Specify the statistic type (e.g., Average, Maximum)
                    'Unit': 'Percent' if metric_name == 'CPUUtilization' else 'Count'  # Specify the unit
                },
                'ReturnData': True
            },
        ],
        StartTime=pd.Timestamp.now() - pd.Timedelta(days=7),  # Adjust the duration as per your requirements
        EndTime=pd.Timestamp.now(),
    )
    
    # Extract the metric data
    metric_data = response['MetricDataResults'][0]['Values']
    timestamps = response['MetricDataResults'][0]['Timestamps']
    
    # Create a pandas DataFrame for the metric data
    df = pd.DataFrame({'Timestamp': timestamps, metric_name: metric_data})
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.set_index('Timestamp', inplace=True)
    data.append(df)

# Merge the metric data into a single DataFrame
merged_data = pd.concat(data, axis=1)

# Plot the metrics in a single chart
plt.figure(figsize=(12, 6))
for metric_name in metric_names:
    plt.plot(merged_data.index, merged_data[metric_name], label=metric_name)
plt.xlabel('Timestamp')
plt.ylabel('Metric Value')
plt.title('Performance Comparison')
plt.legend()
plt.grid(True)
plt.show()